In [1]:
import pandas as pd
import re
import string
import nltk
from nltk.corpus import stopwords
# nltk.download("stopwords")

In [2]:
df = pd.read_csv("../dataset/estee_lauder.csv")

In [1]:
df.head()["search_name"]

NameError: name 'df' is not defined

In [3]:
english_stopwords = stopwords.words("english")
def clean_message(text):
    '''
    Input:
        text: a string containing a text.
    Output:
        message_cleaned: a processed text. 

    '''

    # Convert words into lower case
    text_cleaned = text.lower()
    
    # Remove links
    text_cleaned = re.sub(r"http\S+", "", text_cleaned)
    text_cleaned = re.sub(r"www.\S+", "", text_cleaned)

    # Remove punctuations
    text_cleaned = "".join([char for char in text_cleaned if char not in string.punctuation])

    # Remove stopwords
    text_cleaned = " ".join([word for word in re.split('\W+', text_cleaned) if word not in english_stopwords])

    return text_cleaned

In [4]:
name_rows = ["brand", "name", "category"]
df["search_name"] = df[name_rows].apply(lambda x: clean_message(" ".join(x.astype("str"))), axis=1)

In [ ]:
df["details"] = df["details"].apply(lambda x: x.replace("\n\n", "; ").replace("\n", " ").replace("  ", " ").replace(" ;", ";"))
df["how_to_use"] = df["how_to_use"].apply(lambda x: x.replace("\n\n", "; ").replace("\n", " ").replace("  ", " ").replace(" ;", ";"))

In [7]:
df.to_csv("../dataset/preprocessed_estee_lauder.csv", index=False)

## View Dataset

In [2]:
import pandas as pd
df = pd.read_csv(filepath_or_buffer="../dataset/preprocessed_estee_lauder.csv")

In [3]:
df["search_name"]

0                 aerin mediterranean honeysuckle perfume
1                                aerin amber musk perfume
2                            aerin rose de grasse perfume
3                                aerin lilac path perfume
4                            aerin wild geranium category
                              ...                        
1045    faced melted mattetallic liquified metallic ma...
1046    faced primed poreless face powder setting spra...
1047        faced mr right retractable brush face brushes
1048            faced damn girl lashes lips set face sets
1049    faced melted chocolate liquid matte eye shadow...
Name: search_name, Length: 1050, dtype: object

In [39]:
df["brand"].value_counts()

CLINIQUE              234
TOM FORD              161
Bumble and bumble     104
Too Faced              81
Jo Malone London       76
Origins                73
Estée Lauder           66
Dr. Jart+              54
Smashbox               54
La Mer                 48
GLAMGLOW               33
Kilian                 32
AERIN                  22
Lab Series For Men     12
Name: brand, dtype: int64

In [18]:
df[df["brand"] == "Estée Lauder"].iloc[5,:]["how_to_use"]

'Suggested Usage:-Shake well.-Smooth the lightweight- moisturizing liquid makeup over skin. Start at center of face and blend outward.-To select your shade- choose your skin’s intensity level (from ultra-light to very deep) and undertone (cool- neutral- or warm).'

## Open AI Summarizer

In [2]:
import pandas as pd
df = pd.read_csv(filepath_or_buffer="../dataset/preprocessed_estee_lauder.csv")

In [38]:
import time

In [41]:
time.sleep(2)

In [69]:
import os
import openai
from dotenv import load_dotenv
import time
import re

load_dotenv()
openai.api_key = os.getenv("OPENAI_API_KEY")

def get_summarized_text(text, type="details"):
    time.sleep(2)
    response = openai.Completion.create(
        model="text-davinci-003",
        prompt=f"Summarize this beauty product {type} in 3 to 5 short sentences so it's easy to understand even for 5th grade:\n\n{text}",
        temperature=0.7,
        max_tokens=150,
        top_p=1.0,
        frequency_penalty=0.0,
        presence_penalty=0.0
    )
    return response.choices[0].text

In [45]:
details_summarized = []
details = list(df["details"])
how_to_use_summarized = []
how_to_use = list(df["how_to_use"])
df_details = pd.DataFrame({"details_summarized": [""] * len(details), "details": details })
df_how_to_use = pd.DataFrame({"how_to_use_summarized": [""] * len(how_to_use), "how_to_use": how_to_use })

In [55]:
n = df_details.shape[0]
for i, row in df_details.iterrows():
    if row["details_summarized"] == "":
        print(f"Working on: {i}/{n}", end="\r")
        row["details_summarized"] = get_summarized_text(row["details"])

In [74]:
df["details_summarized"] = df_details["details_summarized"].apply(lambda x: re.sub(".*\n\n", "", x))

In [75]:
df.to_csv("../dataset/preprocessed_estee_lauder.csv", index=False)

In [77]:
n = df_how_to_use.shape[0]
for i, row in df_how_to_use.iterrows():
    if row["how_to_use_summarized"] == "":
        if row["how_to_use"] != "no instructions":
            print(f"Working on: {i}/{n}", end="\r")
            row["how_to_use_summarized"] = get_summarized_text(row["how_to_use"], type="instructions")
        else:
            row["how_to_use_summarized"] = "no instructions"

In [79]:
df["how_to_use_summarized"] = df_how_to_use["how_to_use_summarized"].apply(lambda x: re.sub(".*\n\n", "", x))

In [84]:
df.head()

,index,brand,category,name,size,rating,number_of_reviews,love,price,value_price,...,details,how_to_use,ingredients,online_only,exclusive,limited_edition,limited_time_offer,search_name,details_summarized,how_to_use_summarized
0,0,AERIN,Perfume,Mediterranean Honeysuckle,1.7 oz/ 50 mL,4.0,91,8000,130.0,130.0,...,Fragrance Family: FloralsScent Type: Fruity Fl...,Suggested Usage:-Fragrance is intensified by t...,Alcohol Denat.- Fragrance- Water- Butylphenyl ...,1,0,0,0,aerin mediterranean honeysuckle perfume,This fragrance is a combination of Italian ber...,"Apply fragrance to your body's warm areas, suc..."
1,1,AERIN,Perfume,Amber Musk,1.7 oz/ 50 mL,4.5,77,5600,130.0,130.0,...,Amber musk blends sensual florals- distinctive...,no instructions,Alcohol Denat.- Amber Musk Fragrance- Water- B...,1,0,0,0,aerin amber musk perfume,Amber Musk is a luxurious fragrance that combi...,no instructions
2,2,AERIN,Perfume,Rose De Grasse,1.7 oz/ 50 mL,5.0,12,1700,235.0,235.0,...,Fragrance Family: Floral; Scent Type: Classic ...,no instructions,unknown,1,0,0,0,aerin rose de grasse perfume,Rose de Grasse is a classic floral beauty prod...,no instructions
3,3,AERIN,Perfume,Lilac Path,1.7 oz/ 50 mL,4.5,63,3800,130.0,130.0,...,Lilac Path weaves a delicate story of crisp gr...,no instructions,Alcohol Denat.- Lilac Path Fragrance- Water- L...,1,0,0,0,aerin lilac path perfume,Lilac Path is a beauty product with a scent of...,no instructions
4,4,AERIN,no category,Wild Geranium,0.27 oz/ 8 mL,0.0,0,108,30.0,30.0,...,Fragrance Family: FloralsScent Type: Fresh Flo...,Suggested Usage:-Fragrance is intensified by t...,Alcohol Denat.- Fragrance (Parfum)- Water\Aqua...,1,0,0,0,aerin wild geranium category,AERIN's Wild Geranium fragrance is a fresh and...,"Apply fragrance to your pulse points - wrists,..."


In [85]:
df.to_csv("../dataset/preprocessed_estee_lauder.csv", index=False)

## View Dataset

In [36]:
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
import pandas as pd
df = pd.read_csv("../preprocessed_estee_lauder.csv")
CHROMEDRIVER_PATH = "./chromedriver"

class SeleniumScrapper:

    def __init__(self):
        self.set_chrome_options()

    def set_chrome_options(self):
        chrome_options = Options()
        prefs = {"profile.default_content_setting_values.notifications" : 2}
        chrome_options.add_experimental_option("prefs",prefs)
        # chrome_options.add_argument('--headless')
        # chrome_options.add_argument('--no-sandbox')
        # chrome_options.add_argument('--disable-dev-shm-usage')
        self.driver = webdriver.Chrome(CHROMEDRIVER_PATH,options=chrome_options)

In [45]:
s = SeleniumScrapper()

/var/folders/29/5gqsxp7n0pq0_m7j85mtnl3c0000gn/T/ipykernel_20723/3196732887.py:22: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  self.driver = webdriver.Chrome(CHROMEDRIVER_PATH,options=chrome_options)


In [46]:
df["image_url"][0]

'https://www.sephora.com/productimages/sku/s1794585-main-zoom.jpg?imwidth=164'

In [47]:
s.driver.get(df["image_url"][0])

In [48]:
s.driver.save_screenshot("test.png")

True

In [43]:
b = s.driver.get_screenshot_as_png()
with open("test.png", "w") as f:
    f.write(b)

TypeError: write() argument must be str, not bytes

In [29]:
import urllib.request

opener = urllib.request.build_opener()
opener.addheaders = [('User-agent', 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_3) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/35.0.1916.47 Safari/537.36')]
urllib.request.install_opener(opener)
urllib.request.urlretrieve(df["image_url"][130], "local-filename.jpg")


KeyboardInterrupt: 

In [10]:
df.head()

,index,brand,category,name,size,rating,number_of_reviews,love,price,value_price,...,details,how_to_use,ingredients,online_only,exclusive,limited_edition,limited_time_offer,search_name,details_summarized,how_to_use_summarized
0,0,AERIN,Perfume,Mediterranean Honeysuckle,1.7 oz/ 50 mL,4.0,91,8000,130.0,130.0,...,Fragrance Family: FloralsScent Type: Fruity Fl...,Suggested Usage:-Fragrance is intensified by t...,"Alcohol Denat., Fragrance, Water, Butylphen...",1,0,0,0,aerin mediterranean honeysuckle perfume,This fragrance is a combination of Italian ber...,"Apply fragrance to your body's warm areas, suc..."
1,1,AERIN,Perfume,Amber Musk,1.7 oz/ 50 mL,4.5,77,5600,130.0,130.0,...,Amber musk blends sensual florals- distinctive...,no instructions,"Alcohol Denat., Amber Musk Fragrance, Water,...",1,0,0,0,aerin amber musk perfume,Amber Musk is a luxurious fragrance that combi...,no instructions
2,2,AERIN,Perfume,Rose De Grasse,1.7 oz/ 50 mL,5.0,12,1700,235.0,235.0,...,Fragrance Family: Floral; Scent Type: Classic ...,no instructions,unknown,1,0,0,0,aerin rose de grasse perfume,Rose de Grasse is a classic floral beauty prod...,no instructions
3,3,AERIN,Perfume,Lilac Path,1.7 oz/ 50 mL,4.5,63,3800,130.0,130.0,...,Lilac Path weaves a delicate story of crisp gr...,no instructions,"Alcohol Denat., Lilac Path Fragrance, Water,...",1,0,0,0,aerin lilac path perfume,Lilac Path is a beauty product with a scent of...,no instructions
4,4,AERIN,no category,Wild Geranium,0.27 oz/ 8 mL,0.0,0,108,30.0,30.0,...,Fragrance Family: FloralsScent Type: Fresh Flo...,Suggested Usage:-Fragrance is intensified by t...,"Alcohol Denat., Fragrance (Parfum), Water\Aq...",1,0,0,0,aerin wild geranium category,AERIN's Wild Geranium fragrance is a fresh and...,"Apply fragrance to your pulse points - wrists,..."


In [7]:
for i in list(df[df["brand"] == "CLINIQUE"]["name"]):
    print(i)

Moisture Surge 72-Hour Auto-Replenishing Hydrator
Dramatically Different Moisturizing Lotion+
Dramatically Different Moisturizing Gel
Liquid Facial Soap
Even Better Foundation Makeup Broad Spectrum SPF 15
Even Better Clinical™ Radical Dark Spot Corrector + Interrupter
Acne Solutions Clinical Clearing Kit
Clarifying Lotion 2
High Impact Mascara
Take The Day Off Cleansing Balm
Beyond Perfecting Foundation + Concealer
Clinique iD™ Custom-Blend Hydrator Collection
All About Eyes™ Rich Eye Cream
Acne Solutions™ Clinical Clearing Gel
Take The Day Off Makeup Remover For Lids, Lashes & Lips
Stay-Matte Sheer Pressed Powder
All About Eyes
Moisture Surge Intense Skin Fortifying Hydrator
Lash Power Mascara Long-Wearing Formula
Acne Solutions Cleansing Foam
Acne Solutions Liquid Makeup Foundation
Exfoliating Scrub
Quickliner for Eyes Intense
Redness Solutions with Probiotic Technology Daily Relief Cream
Clarifying Lotion 3
Great Skin, Great Deal Set for Dry Combination Skin
Superpowder Double Face 

## Correct Ingredients

In [21]:
def correct_ingredients(text):
    text_token = [ing for ing in text.split("-") if len(ing) < 50]
    return ", ".join(text_token)

df["ingredients"] = df["ingredients"].apply(lambda x: correct_ingredients(x))

In [24]:
df.to_csv("../dataset/preprocessed_estee_lauder.csv", index=False)

In [26]:
df["details_summarized"][0]

'This fragrance is a combination of Italian bergamot, honeysuckle, grapefruit, lily of the valley, gardenia, and jasmine sambac absolute. It is inspired by the beauty and style of the Mediterranean, and aims to evoke lush flowers and shimmering beaches. It is a light and refreshing scent that will make you feel like you are on a sunny Mediterranean beach.'